# Custom Container Types

It's common in Python to create classes that contain and manage data. Python comes with powerful built-in container types for managing data such as: `dictionaries`, `lists`, `sets` and `tuples`.

When you create classes for sequences such as `lists` it's common to sub-class the built-in `list` type. This way you have access to all the functionality of the `list` type with your own additional functionality built on top.  Consider the following example where we add functionality to find the duplicates in a list:

In [1]:
class DeDupList(list):
    def __init__(self, list_members):
        super().__init__(list_members)
    
    def dups(self):
        d = {}
        for item in self:
            d[item] = d.get(item, 0) + 1
        return [item for item, count in d.items() if count > 1]

In [2]:
l = DeDupList(['a', 'b', 'c', 'a', 'c'])

In [3]:
l.dups()

['a', 'c']

The `DeDupList` class sub-classes the `list` type, so we should still have access to list methods like `len`, `index`, `append`, `pop`, etc...

In [4]:
len(l)

5

In [5]:
l.index('b')

1

In [6]:
l.append('z')

In [7]:
l

['a', 'b', 'c', 'a', 'c', 'z']

In [8]:
l.dups()

['a', 'c']

In [9]:
l.append('z')
l

['a', 'b', 'c', 'a', 'c', 'z', 'z']

In [10]:
l.dups()

['a', 'c', 'z']

In [11]:
l.pop()

'z'

In [12]:
l.dups()

['a', 'c']

Now, imagine we want to create a `GuestList` class which will manage the people on the list:

In [13]:
class GuestList(object):
    def __init__(self, guest_list):
        self.guest_list = guest_list

Can we find the first person on the guest list?

In [14]:
guests = GuestList(['Person A', 'Person B', 'Person C'])
guests[0]

TypeError: 'GuestList' object does not support indexing

Looks like we can't, so how can implement this?

When you call `guests[0]` on a sequence Python is essentially calling `guests.__getitem__(0)`. This function is not defined in our class so we have to define it. This is redundant I know, because we can just use the `list` type to do all this. For arguments sake let’s implement this sequence ourselves.

In [15]:
class SequenceGuestList(GuestList):
    
    def __getitem__(self, i):
        return self.guest_list[i]

In [16]:
guests = SequenceGuestList(['Person A', 'Person B', 'Person C'])

In [17]:
guests[0]

'Person A'

The issue we have now is that we have not implemented other sequence functionality such as `len`. This method requires the built-in method `__len__` to be defined. We could define this in our class but this is still would not be enough, we need to define other sequence methods like `index` and `count` which a programmer would expect to be implemented. Defining a custom container is not as easy as it looks.

Luckily, Python already has our backs covered. We can use the `collections.abc` module to define a set of abstract base classes that provide all of the typical methods on a given container type. All you have to do is subclass one of the abstract base classes and Python will complain if you forget to implement something.


In [18]:
from collections.abc import Sequence

class SequenceGuestList(GuestList, Sequence):
    
    def __getitem__(self, i):
        return self.guest_list[i]

Let’s create an instance of the new `SequenceGuestList` which has subclassed the `Sequence` abstract class. This should throw and error stating that `__len__` is not defined.

In [19]:
guests = SequenceGuestList(['Person A', 'Person B', 'Person C'])

TypeError: Can't instantiate abstract class SequenceGuestList with abstract methods __len__

Great, it does, let’s define the `__len__` method and create a new instance:

In [20]:
from collections.abc import Sequence

class SequenceGuestList(GuestList, Sequence):
    
    def __getitem__(self, i):
        return self.guest_list[i]
    
    def __len__(self):
        return len(self.guest_list)

guests = SequenceGuestList(['Person A', 'Person B', 'Person C'])

No complaints which is good! But what about the other methods like `index` and `count`? Why didn’t Python complain about them?

In [21]:
guests.index('Person B')

1

In [22]:
guests.count('Person C')

1

Python doesn’t need us to define these methods because it can work them out from using `__len__` and `__getitem__` which have been defined. So by defining these two methods we get extra functionality for free! Making it much easier to define custom container types.